# Example 3-15: Performing an *IAU-76/FK5* Reduction
### _Fundamentals of Astrodynamics and Applications_, 5th Ed., 2022, p. 231-232

This notebook demonstrates using the IAU-76/FK5 reductions to transform from the terrestrial (ITRF) coordinate system to the celestial (GCRF) coordinate system.

## Install and Import Libraries
---

First, install `valladopy` if it doesn't already exist in your environment:

In [1]:
!pip install valladopy

Import `os` and the relevant `valladopy` modules:

In [2]:
import os

import valladopy.constants as const
from valladopy.astro.time.data import iau80in
from valladopy.astro.time.frame_conversions import ecef2eci
from valladopy.mathtime.julian_date import convtime

## Problem Definition
---

GIVEN: $\quad\overrightarrow{r}_{ITRF} = -1033.479383 \; \hat{I} \; + 7901.2952754 \; \hat{J} \; + 6380.3565958 \; \hat{K}$ km<br>
       $\quad\quad\quad\quad\overrightarrow{v}_{ITRF} = -3.22563652 \; \hat{I} \; - 2.87245145 \; \hat{J} \; + 5.531924446 \; \hat{K}$ km/s<br>
FIND:&emsp;$\quad\overrightarrow{r}_{GCRF},\ \overrightarrow{v}_{GCRF}$ on April 6, 2004, 07:51:28.386009 UTC

In [3]:
# ITRF (i.e. ECEF) state vectors
r_ecef = [-1033.479383, 7901.2952754, 6380.3565958]  # km
v_ecef = [-3.22563652, -2.87245145, 5.531924446]     # km/s
a_ecef = [0, 0, 0]                                   # dummy acceleration, km/s²

# Date
year = 2004
month = 4
day = 6
hour = 7
minute = 51
second = 28.386009
timezone = 0  # hours offset from UTC

## Solution
---

We first determine the time quantities prior to starting the problem, the same as we did in [Example 3-14](Example_3-14.ipynb), where:

$$
\Delta AT = 32.0^{\text{S}}
$$

And the EOP quantities are taken from the IERS's EOPC04 [final data](https://datacenter.iers.org/data/latestVersion/finals.all.iau2000.txt), with some converted from arcseconds to radians: 

In [4]:
# Leap seconds
dat = 32                              # seconds

# EOP quantities
dut1 = -0.4399619                     # seconds
xp = -0.140682 * const.ARCSEC2RAD     # radians
yp = 0.333309 * const.ARCSEC2RAD      # radians
lod = 0.0015563                       # seconds
ddpsi = -0.052195 * const.ARCSEC2RAD  # radians
ddeps = -0.003875 * const.ARCSEC2RAD  # radians

(These are the same non-interpolated values as in the previous example.)

Next, convert UTC to various time systems using the `convtime` routine (see [Example 3-7](Example_3-7.ipynb) for more details on this process). We just need the Julian centuries of TT and the Julian date of UT1 for the final transformation:

In [5]:
_, _, jdut1, jdut1frac, _, _, _, ttt, *_ = convtime(year, month, day, hour, minute, second, timezone, dut1, dat)

print(f'Julian centuries of TT:\t{ttt}')

Julian centuries of TT:	0.04262363188899416


In order to convert the fixed vectors (ITRF/ECEF) to inertial vectors (GCRF/ECI), we need to obtain the nutation matrices for the reduction calculations. The `iau80in` routine requires the nutation data to be present in a given data directory.

Replace the following data directory definition with your preferred location. We will use the relative path from this notebook, assuming the same structure as [repository](https://github.com/CelesTrak/fundamentals-of-astrodynamics) to reach the `datalib` directory. The file name is hardcoded for now but is planned to be flexible in the future — make sure these are included in your data directory.

In [6]:
data_dir = "../../../datalib/"  # relative data path

iau80arr = iau80in(data_dir)

print(f'IAU 80 nutation matrices keys:\n{iau80arr.__dict__.keys()}\n')

IAU 80 nutation matrices keys:
dict_keys(['iar80', 'rar80'])



**Algorithm 24** summarizes the process of transforming the state vectors between the fixed and inertial coordinate systems using IAU-76/FK5 reduction.

The precession matrix $P$ is:

$$
[\mathbf{P}] = \text{ROT3}(\zeta) \cdot \text{ROT2}(-\Theta) \cdot \text{ROT3}(z)
$$

where:

$$
\begin{aligned}
\zeta &= 2306.2181^{\prime\prime} \ T_{TT} + 0.30188 \ T_{TT}^2 + 0.017998 \ T_{TT}^3 \\
\\
\Theta &= 2004.3109^{\prime\prime} \ T_{TT} - 0.42665 \ T_{TT}^2 - 0.041833 \ T_{TT}^3 \\
\\
z &= 2306.2181^{\prime\prime} \ T_{TT} + 1.09468 \ T_{TT}^2 + 0.018203\ T_{TT}^3 \\
\end{aligned}
$$

The nutation matrix is:

$$
[\mathbf{N}] = \text{ROT1}(-\epsilon_{1980}) \cdot \text{ROT3}(\Delta\Psi_{1980}) \cdot \text{ROT1}(\epsilon_{1980})
$$

where the mean obliquity of the ecliptic $\epsilon_{1980}$ is:

$$
\epsilon_{1980} = 84,381.448^{\prime\prime} - 46.815 \ T_{TT} - 0.00059 \ T_{TT}^2 + 0.001813 \ T_{TT}^3
$$

and $\Delta\Psi_{1980}$ is the nutation in longitude and can be found with **Equation 3-86** (see p. 227 for details).

The sidereal time rotation matrix $R$ is found with:

$$
[\mathbf{R}] = \text{ROT3}(-\theta_{GAST1982})
$$

where $\theta_{GAST1982}$ is the Greenwich apparent sidereal time (see **Equation 3-82** on p. 225 for details):

The rotation matrix $W$ from ITRF to TIRS (to account for polar motion) is:

$$
[\mathbf{W}] = \text{ROT1}(y_p) \cdot \text{ROT2}(x_p) \approx
\left[\begin{array}{ccc}
1 & 0 & -x_p \\
0 & 1 & y_p \\
x_p & -y_p & 1
\end{array} \right]\\
$$

The GCRF (inertial) state vectors are then computed as:

$$
\begin{aligned}
\vec{r}_{GCRF} &= [\mathbf{P}][\mathbf{N}][\mathbf{R}][\mathbf{W}]\vec{r}_{ITRF} \\
\\
\vec{v}_{GCRF} &= [\mathbf{P}][\mathbf{N}][\mathbf{R}] \left\{ [\mathbf{W}]\vec{v}_{ITRF} + \vec{\omega}_\oplus \times \vec{r}_{PEF} \right\}
\end{aligned}
$$

where $\vec{\omega}_\oplus$ is the Earth rotation vector (with $LOD$ as the length of day in seconds):

$$
\vec{\omega}_\oplus = \left[ 0,\ 0,\ 7.292115146706979 \times 10^{-5} \right]\left\{ 1 - \frac{LOD}{86400} \right\} \ \text{rad/s}
$$

and $\vec{r_{PEF}}$ is simply:

$$
\vec{r_{PEF}} = [\mathbf{W}]_{\text{ITRF-PEF}} \ \ \vec{r_{ITRF}}
$$

(Note that the PEF frame here is analogous to the TIRS frame in the IAU-2006/2000 reduction calculations)

All of this is handled internally by the `ecef2eci` routine, which applies the reduction using the provided EOP and nutation data:

In [7]:
r_eci, v_eci, _ = ecef2eci(r_ecef, v_ecef, a_ecef, ttt, jdut1+jdut1frac, lod, xp, yp, ddpsi, ddeps, iau80arr)

print('GCRF (ECI) state vectors:\n')
print(f'r_gcrf:\t{r_eci}\tkm')
print(f'v_gcrf:\t{v_eci}\t\tkm/s')

GCRF (ECI) state vectors:

r_gcrf:	[5102.50895792 6123.01140072 6378.13692819]	km
v_gcrf:	[-4.74322015  0.79053649  5.53375573]		km/s


If we had chosen not to use the $\delta\Delta\Psi_{1980}$ and $\delta\Delta\epsilon_{1980}$ corrections, we find a vector that is close to the GCRF, but about a meter different:

In [8]:
ddpsi, ddeps = 0, 0
r_eci, v_eci, _ =  ecef2eci(r_ecef, v_ecef, a_ecef, ttt, jdut1+jdut1frac, lod, xp, yp, ddpsi, ddeps, iau80arr)

print('GCRF (ECI) state vectors:\n')
print(f'r_gcrf:\t{r_eci}\tkm')
print(f'v_gcrf:\t{v_eci}\t\tkm/s')

GCRF (ECI) state vectors:

r_gcrf:	[5102.50960002 6123.01152001 6378.13629999]	km
v_gcrf:	[-4.74321959  0.79053659  5.53375619]		km/s
